In [ ]:
#!pip install -U tensorflow
!pip install tensorflow_text==2.8.2
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
import pandas as pd

# Try different encodings until you find the right one
# Common encodings to try: 'latin-1', 'ISO-8859-1', 'cp1252'
try:
    df = pd.read_csv("/content/sample_data/spam.csv", encoding='latin-1')
    # If 'latin-1' doesn't work, try other encodings
except UnicodeDecodeError:
    df = pd.read_csv("/content/sample_data/spam.csv", encoding='ISO-8859-1')
    # If 'ISO-8859-1' doesn't work, try other encodings

df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.groupby('v1').describe()

v2                                                                 \
     count unique                                                top freq   
v1                                                                          
ham   4825   4516                             Sorry, I'll call later   30   
spam   747    653  Please call our customer service representativ...    4   

     Unnamed: 2                                                            \
          count unique                                                top   
v1                                                                          
ham          45     39   bt not his girlfrnd... G o o d n i g h t . . .@"   
spam          5      4                                        PO Box 5249   

          Unnamed: 3                                    Unnamed: 4         \
     freq      count unique                    top freq      count unique   
v1                                                                          
ham     3         10      9                     GE    2          6      5   
spam    2          2      1   MK17 92H. 450Ppw 16"    2          0      0   

                    
          top freq  
v1                  
ham   GNT:-)"    2  
spam      NaN  NaN

In [ ]:
df['v1'].value_counts()

,count
v1,
ham,4825
spam,747


In [ ]:
df_spam = df[df['v1']=='spam']
df_spam.shape
df_ham = df[df['v1']=='ham']
df_ham.shape

(4825, 5)

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 5)

In [ ]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 5)

In [ ]:
df_balanced['v1'].value_counts()

,count
v1,
ham,747
spam,747


In [ ]:
df_balanced['spam']=df_balanced['v1'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,spam
2043,spam,4mths half price Orange line rental & latest c...,NaN,NaN,NaN,1
1832,ham,What time is ur flight tmr?,NaN,NaN,NaN,0
3700,ham,Shall i get my pouch?,NaN,NaN,NaN,0
3513,ham,I always chat with you. In fact i need money c...,NaN,NaN,NaN,0
2704,spam,FreeMsg: Fancy a flirt? Reply DATE now & join ...,NaN,NaN,NaN,1


In [ ]:
df_balanced['spam']=df_balanced['v1'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,spam
4392,spam,RECPT 1/3. You have ordered a Ringtone. Your o...,NaN,NaN,NaN,1
5367,ham,Just trying to figure out when I'm suppose to ...,NaN,NaN,NaN,0
3289,ham,My tuition is at 330. Hm we go for the 1120 to...,NaN,NaN,NaN,0
3295,ham,Hi there. We have now moved in2 our pub . Woul...,NaN,NaN,NaN,0
5142,ham,Now that you have started dont stop. Just pray...,NaN,NaN,NaN,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['v2'],df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
X_train.head(4)

,v2
867,PRIVATE! Your 2003 Account Statement for 07808...
2548,Honestly i've just made a lovely cup of tea an...
823,25p 4 alfie Moon's Children in need song on ur...
380,I guess that's why you re worried. You must kn...


In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435172 , -0.5132728 , -0.88845754, ..., -0.7474889 ,
        -0.75314754,  0.91964495],
       [-0.87208337, -0.5054394 , -0.94446665, ..., -0.85847485,
        -0.71745324,  0.88082963]], dtype=float32)>

In [ ]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.760692  , -0.14219391,  0.49604604, ...,  0.42165324,
        -0.5322142 ,  0.8031219 ],
       [-0.8602322 , -0.21242957,  0.4915689 , ...,  0.3979803 ,
        -0.60506296,  0.84471667],
       [-0.7128861 , -0.15463915,  0.38401663, ...,  0.3527872 ,
        -0.5099134 ,  0.73474085],
       [-0.8253347 , -0.35550573, -0.5906969 , ..., -0.01613709,
        -0.6141758 ,  0.87230283],
       [-0.7504134 , -0.2681261 , -0.26689658, ...,  0.02839418,
        -0.5938099 ,  0.7974987 ],
       [-0.78544396, -0.29949713,  0.41027236, ...,  0.5222534 ,
        -0.4957357 ,  0.81507534]], dtype=float32)>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[4]])

array([[0.89336324]], dtype=float32)

In [ ]:
len(X_train)

1120

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 636s 18s/step - loss: 0.5872 - accuracy: 0.7116 - precision: 0.7061 - recall: 0.7250
Epoch 2/10
35/35 [==============================] - 612s 18s/step - loss: 0.4691 - accuracy: 0.8259 - precision: 0.8230 - recall: 0.8304
Epoch 3/10
35/35 [==============================] - 612s 18s/step - loss: 0.4144 - accuracy: 0.8562 - precision: 0.8494 - recall: 0.8661
Epoch 4/10
35/35 [==============================] - 609s 17s/step - loss: 0.3715 - accuracy: 0.8705 - precision: 0.8535 - recall: 0.8946
Epoch 5/10
35/35 [==============================] - 623s 18s/step - loss: 0.3402 - accuracy: 0.8795 - precision: 0.8657 - recall: 0.8982
Epoch 6/10
35/35 [==============================] - 611s 17s/step - loss: 0.3171 - accuracy: 0.8938 - precision: 0.8835 - recall: 0.9071
Epoch 7/10
35/35 [==============================] - 625s 18s/step - loss: 0.3020 - accuracy: 0.8973 - precision: 0.8790 - recall: 0.9214
Epoch 8/10
35/35 [=======================

In [ ]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 213s 17s/step - loss: 0.2754 - accuracy: 0.9064 - precision: 0.9000 - recall: 0.9144


[0.2753761410713196,
 0.9064171314239502,
 0.8999999761581421,
 0.9144384860992432]

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.7249197 ],
       [0.79679346],
       [0.786399  ],
       [0.15657091],
       [0.087358  ]], dtype=float32)